## National Disaster Data

### Data Source: https://www.fema.gov/openfema-data-page/disaster-declarations-summaries-v2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt


In [ ]:
df = pd.read_csv('/work/assets/DisasterDeclarationsSummaries.csv')

county_mapper = pd.read_csv('/work/cleaned-csvs/us_counties.csv')

In [ ]:
df = df[df['fyDeclared'].between(2014,2019)]
df['incidentType'].value_counts()

Hurricane           2308
Severe Storm(s)     2156
Flood               1735
Fire                 372
Severe Ice Storm     353
Snow                 222
Tornado               83
Coastal Storm         45
Mud/Landslide         27
Typhoon               20
Earthquake            10
Chemical               9
Dam/Levee Break        3
Volcano                2
Toxic Substances       1
Name: incidentType, dtype: int64

In [ ]:
df[df['state']=='CA']

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,incidentEndDate,disasterCloseoutDate,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,hash,lastRefresh,id
42094,EM-3381-CA,3381,CA,EM,2017-02-14T19:20:00.000Z,2017,Dam/Levee Break,POTENTIAL FAILURE OF THE EMERGENCY SPILLWAY AT...,0,0,...,2017-02-23T23:59:00.000Z,NaN,6,7,99007,Butte (County),17013,39dd0eec3d8314ed7e053d9b27c0f343c45b77a0,2021-07-13T23:02:04.122Z,60c3b85ba0ee349d710a647b
42095,EM-3381-CA,3381,CA,EM,2017-02-14T19:20:00.000Z,2017,Dam/Levee Break,POTENTIAL FAILURE OF THE EMERGENCY SPILLWAY AT...,0,0,...,2017-02-23T23:59:00.000Z,NaN,6,101,99101,Sutter (County),17013,5db5ce5334f8cc8ae905535b15e54346db3e64bd,2021-07-13T23:02:04.122Z,60c3b85ba0ee349d710a6483
42098,EM-3381-CA,3381,CA,EM,2017-02-14T19:20:00.000Z,2017,Dam/Levee Break,POTENTIAL FAILURE OF THE EMERGENCY SPILLWAY AT...,0,0,...,2017-02-23T23:59:00.000Z,NaN,6,115,99115,Yuba (County),17013,a64bc58946fb76c1548a8427a10e1f75831cdf3a,2021-07-13T23:02:04.123Z,60c3b85ba0ee349d710a6492
42615,EM-3396-CA,3396,CA,EM,2017-12-08T10:00:00.000Z,2018,Fire,WILDFIRES,0,0,...,2017-12-29T23:59:00.000Z,NaN,6,83,99083,Santa Barbara (County),17146,651480df722310f0f2371cf5ec4639f8eaefcbb0,2021-07-13T23:02:04.492Z,60c3b85ba0ee349d710a6f9b
42616,EM-3396-CA,3396,CA,EM,2017-12-08T10:00:00.000Z,2018,Fire,WILDFIRES,0,0,...,2017-12-29T23:59:00.000Z,NaN,6,37,99037,Los Angeles (County),17146,da5bf56e9f9817c0483f129ce126552f471d91d6,2021-07-13T23:02:04.490Z,60c3b85ba0ee349d710a6f8a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61737,FM-5262-CA,5262,CA,FM,2018-07-28T22:49:00.000Z,2018,Fire,MENDOCINO FIRE COMPLEX,0,0,...,2018-08-20T23:59:00.000Z,NaN,6,0,34659,Hopland Rancheria (Indian Reservation),18070,c8e76dcc5889ec7d46ef9c2161678a1162237509,2021-07-13T23:02:24.627Z,60c3b876a0ee349d710bfb65
61759,FM-5280-CA,5280,CA,FM,2018-11-09T05:50:00.000Z,2019,Fire,WOOLSEY FIRE,0,0,...,NaN,2019-02-06T23:59:00.000Z,6,37,99037,Los Angeles (County),18124,fce8164d6a9502ad6176ea57ad1051a690dcfce7,2021-07-19T14:02:23.214Z,60c3b876a0ee349d710bfbb4
61837,FM-5278-CA,5278,CA,FM,2018-11-08T14:38:00.000Z,2019,Fire,CAMP FIRE,0,0,...,NaN,NaN,6,7,99007,Butte (County),18125,e9227ef9709f2042e0b5fbc7b054401f64e6ac99,2021-08-17T11:41:56.946Z,60c3b876a0ee349d710bfbd8
61855,FM-5280-CA,5280,CA,FM,2018-11-09T05:50:00.000Z,2019,Fire,WOOLSEY FIRE,0,0,...,NaN,2019-02-06T23:59:00.000Z,6,111,99111,Ventura (County),18124,c909ea3bf245f0d0148e0463663936a37600a5a4,2021-07-19T14:02:23.214Z,60c3b876a0ee349d710bfbdf


In [ ]:
df = df[['state', 'fyDeclared', 'incidentType',  
       'fipsStateCode', 'fipsCountyCode', 
       'designatedArea',]]

# all but one disaster ends in the same year it started in
#for each value of incident type, creates a new column with 0/1 values
df = pd.get_dummies(df, columns=['incidentType'])

df.columns

rename_dict = {'fyDeclared':'year', 'fipsStateCode':'state_code', 'fipsCountyCode':'county_code',
       'designatedArea':'county', 'incidentType_Chemical':'chemical', 
       'incidentType_Coastal Storm':'coastal_storm',
       'incidentType_Dam/Levee Break':'dam_levee_break', 'incidentType_Earthquake':'earthquake',
       'incidentType_Fire':'fire', 'incidentType_Flood':'flood', 'incidentType_Hurricane':'hurricane',
       'incidentType_Mud/Landslide':'mud_landslide', 'incidentType_Severe Ice Storm':'severe_ice_storm',
       'incidentType_Severe Storm(s)':'severe_storm', 'incidentType_Snow':'snow',
       'incidentType_Tornado':'tornado', 'incidentType_Toxic Substances':'toxic_substances',
       'incidentType_Typhoon':'typhoon', 'incidentType_Volcano':'volcano'}
df.rename(columns = rename_dict, inplace = True)

In [ ]:
df = df.groupby(['state_code','county_code','year']).sum()

df.reset_index(inplace = True)

In [ ]:
# sanity check to ensure this data has same counties as ACS data
df_merged = pd.merge(df,county_mapper, how = 'left', left_on = ['state_code','county_code'],right_on = ['state','county'] )

In [ ]:
#There are 59 rows of data that can't be mapped onto the mapper

# The areas coded with 0 as county are reservations. This represents about 34 rows.
# the remaining 25 rows appear not to map onto a county that currently exists. 


# For example, this county changed designations https://www.economy.com/support/blog/buffet.aspx?did=50094FC4-C32C-4CCA-862A-264BC890E13B
# df[(df['state_code']==51) & (df['county_code']==515)]


# Any with code 60 represents American Samoa (this is about 15 of the 25)
#df[(df['state_code']==60) ]
df_merged[(df_merged['county'].isna())]

df_merged = df_merged.dropna()

df_merged = df_merged.iloc[:,2:]

In [ ]:
#Check CA df to see how many fires are taking place

df_ca = df_merged[df_merged['state_name']=='california']

df_ca[df_ca['fire'] !=0]

,year,chemical,coastal_storm,dam_levee_break,earthquake,fire,flood,hurricane,mud_landslide,severe_ice_storm,severe_storm,snow,tornado,toxic_substances,typhoon,volcano,state,county,county_name,state_name
389,2015,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6.0,5.0,amador,california
392,2017,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,6.0,7.0,butte,california
393,2018,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,6.0,7.0,butte,california
394,2019,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,6.0,7.0,butte,california
395,2015,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,6.0,9.0,calaveras,california
403,2014,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6.0,17.0,el dorado,california
406,2016,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6.0,19.0,fresno,california
412,2014,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6.0,29.0,kern,california
413,2016,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,6.0,29.0,kern,california
417,2015,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,6.0,33.0,lake,california


In [ ]:
df_merged.to_csv('/work/cleaned-csvs/fema.csv')

In [ ]:
# #df[df['state']=='CA']['fyDeclared'].value_counts()
# df['designatedArea'].value_counts()
# #df[(df['state']=='CA') & df['fyDeclared']]['designatedArea'].value_counts()
# df[(df['state']=='CA') ]['designatedArea'].value_counts()
# #df[(df['state']=='CA') & df['fyDeclared'].between(2014,2018)].head()#['designatedArea'].value_counts()

# df[(df['state']=='CA') & df['fyDeclared'].between(2014,2018)]['declarationTitle'].value_counts()
# ca_df = df[(df['state']=='CA') & df['fyDeclared'].between(2014,2018)]
# ca_df['incidentType'].value_counts()

In [ ]:
df_merged.columns

Index(['year', 'chemical', 'coastal_storm', 'dam_levee_break', 'earthquake',
       'fire', 'flood', 'hurricane', 'mud_landslide', 'severe_ice_storm',
       'severe_storm', 'snow', 'tornado', 'toxic_substances', 'typhoon',
       'volcano', 'state', 'county', 'county_name', 'state_name'],
      dtype='object')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6c76417-5fde-42f3-8920-755838dec3fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>